In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Reference
# https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2
# https://github.com/scrapinghub/python-crfsuite/blob/master/examples/CoNLL%202002.ipynb

host = 0
# host = 4

if host == 0:
    path = '/Users/aron/Documents/GitHub/Perfume/2_NLP'
elif host == 4:
    path = r'/home/rserver/Data_Mining/personal_workspace/yz/Lab/CkipTagger/'

In [2]:
# 新增工作資料夾
path_resource = path + '/Resource'
path_function = path + '/Function'
path_temp = path + '/Temp'
path_export = path + '/Export'

In [5]:
df = pd.read_csv(path_resource + '/ner_dataset.csv', encoding = "ISO-8859-1")
df = df[:10000]
df.head()
df.isnull().sum()

Sentence #    9543
Word             0
POS              0
Tag              0
dtype: int64

In [6]:
df

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
9995,NaN,war,NN,O
9996,NaN,crimes,NNS,O
9997,NaN,tribunal,NN,O
9998,NaN,in,IN,O


## Data Preprocessing

In [7]:
df = df.fillna(method='ffill')
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()

(457, 2746, 17)

In [8]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-art,28
1,B-eve,10
2,B-geo,244
3,B-gpe,303
4,B-nat,5
5,B-org,176
6,B-per,160
7,B-tim,149
8,I-art,20
9,I-eve,10


In [9]:
X = df.drop('Tag', axis=1)
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
y = df.Tag.values
classes = np.unique(y)
classes = classes.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)
X_train.shape, y_train.shape

((6700, 3242), (6700,))

## Out-of-core Algorithms
### Perceptron

In [10]:
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
per.partial_fit(X_train, y_train, classes)

-- Epoch 1
-- Epoch 1-- Epoch 1

-- Epoch 1-- Epoch 1-- Epoch 1


-- Epoch 1-- Epoch 1

Norm: 5.48, NNZs: 27, Bias: -2.000000, T: 6700, Avg. loss: 0.002239
Total training time: 0.05 seconds.
Norm: 8.77, NNZs: 65, Bias: -3.000000, T: 6700, Avg. loss: 0.005224
Total training time: 0.05 seconds.
Norm: 22.11, NNZs: 339, Bias: -5.000000, T: 6700, Avg. loss: 0.041343Norm: 17.83, NNZs: 166, Bias: -2.000000, T: 6700, Avg. loss: 0.012985
Total training time: 0.05 seconds.

Total training time: 0.04 seconds.
-- Epoch 1Norm: 18.73, NNZs: 241, Bias: -5.000000, T: 6700, Avg. loss: 0.024776

Total training time: 0.05 seconds.
-- Epoch 1Norm: 5.10, NNZs: 18, Bias: -2.000000, T: 6700, Avg. loss: 0.001343
Norm: 19.57, NNZs: 269, Bias: -3.000000, T: 6700, Avg. loss: 0.028358
Total training time: 0.05 seconds.
Norm: 26.00, NNZs: 389, Bias: -4.000000, T: 6700, Avg. loss: 0.040299-- Epoch 1

Total training time: 0.05 seconds.

-- Epoch 1-- Epoch 1
Total training time: 0.05 seconds.

-- Epoch 1-- Epoch 1
--

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   4 out of  17 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   6 out of  17 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   8 out of  17 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  17 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  12 out of  17 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  14 out of  17 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:    0.1s finished


Perceptron(max_iter=5, n_jobs=-1, verbose=10)

In [11]:
new_classes = classes.copy()
new_classes.pop()
new_classes

['B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim']

In [12]:
print(classification_report(y_pred=per.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

       B-art       0.24      0.44      0.31         9
       B-eve       0.00      0.00      0.00         3
       B-geo       0.87      0.19      0.31        69
       B-gpe       0.59      0.71      0.64       102
       B-nat       0.00      0.00      0.00         0
       B-org       0.34      0.75      0.46        63
       B-per       0.94      0.41      0.58        41
       B-tim       0.59      0.94      0.73        52
       I-art       0.33      0.20      0.25        10
       I-eve       0.00      0.00      0.00         3
       I-geo       0.00      0.00      0.00        11
       I-gpe       1.00      0.17      0.29         6
       I-nat       0.00      0.00      0.00         1
       I-org       0.43      0.21      0.29        47
       I-per       0.69      0.27      0.39        66
       I-tim       0.00      0.00      0.00         4

   micro avg       0.51      0.48      0.49       487
   macro avg       0.38   

/Users/aron/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aron/opt/anaconda3/lib/

### Linear classifiers with SGD training

In [13]:
sgd = SGDClassifier()
sgd.partial_fit(X_train, y_train, classes)

SGDClassifier()

In [14]:
print(classification_report(y_pred=sgd.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

       B-art       0.75      0.33      0.46         9
       B-eve       0.00      0.00      0.00         3
       B-geo       0.29      0.84      0.43        69
       B-gpe       0.77      0.60      0.67       102
       B-nat       0.00      0.00      0.00         0
       B-org       0.57      0.44      0.50        63
       B-per       0.53      0.46      0.49        41
       B-tim       0.87      0.79      0.83        52
       I-art       0.40      0.20      0.27        10
       I-eve       0.00      0.00      0.00         3
       I-geo       0.00      0.00      0.00        11
       I-gpe       0.67      0.33      0.44         6
       I-nat       0.00      0.00      0.00         1
       I-org       0.60      0.13      0.21        47
       I-per       0.83      0.08      0.14        66
       I-tim       0.19      1.00      0.32         4

   micro avg       0.50      0.47      0.48       487
   macro avg       0.40   

/Users/aron/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aron/opt/anaconda3/lib/

In [15]:
### Naive Bayes classifier for multinomial models

In [16]:
nb = MultinomialNB(alpha=0.01)
nb.partial_fit(X_train, y_train, classes)

MultinomialNB(alpha=0.01)

In [17]:
print(classification_report(y_pred=nb.predict(X_test), y_true=y_test, labels = new_classes))

              precision    recall  f1-score   support

       B-art       0.11      0.33      0.16         9
       B-eve       0.00      0.00      0.00         3
       B-geo       0.62      0.49      0.55        69
       B-gpe       0.64      0.73      0.68       102
       B-nat       0.00      0.00      0.00         0
       B-org       0.58      0.49      0.53        63
       B-per       0.40      0.51      0.45        41
       B-tim       0.67      0.85      0.75        52
       I-art       0.29      0.20      0.24        10
       I-eve       0.20      0.33      0.25         3
       I-geo       0.25      0.36      0.30        11
       I-gpe       0.25      0.50      0.33         6
       I-nat       0.00      0.00      0.00         1
       I-org       0.48      0.62      0.54        47
       I-per       0.60      0.39      0.48        66
       I-tim       0.06      0.25      0.10         4

   micro avg       0.51      0.56      0.53       487
   macro avg       0.32   

/Users/aron/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aron/opt/anaconda3/lib/

### Passive Aggressive Classifier

In [18]:
pa = PassiveAggressiveClassifier()
pa.partial_fit(X_train, y_train, classes)

PassiveAggressiveClassifier()

In [19]:
print(classification_report(y_pred=pa.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00         9
       B-eve       0.25      0.33      0.29         3
       B-geo       0.74      0.29      0.42        69
       B-gpe       0.57      0.79      0.67       102
       B-nat       0.00      0.00      0.00         0
       B-org       0.45      0.51      0.48        63
       B-per       0.20      0.66      0.31        41
       B-tim       0.95      0.73      0.83        52
       I-art       0.00      0.00      0.00        10
       I-eve       0.00      0.00      0.00         3
       I-geo       0.60      0.27      0.37        11
       I-gpe       1.00      0.50      0.67         6
       I-nat       0.00      0.00      0.00         1
       I-org       1.00      0.11      0.19        47
       I-per       0.64      0.32      0.42        66
       I-tim       1.00      0.25      0.40         4

   micro avg       0.50      0.48      0.49       487
   macro avg       0.46   

/Users/aron/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aron/opt/anaconda3/lib/

## Conditional Random Fields (CRFs)
### sklearn-crfsuite

In [20]:
# import sklearn_crfsuite
# from sklearn_crfsuite import scorers
# from sklearn_crfsuite import metrics
from collections import Counter

In [21]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences

In [22]:
getter

### Feature Extraction

In [23]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
        
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]


### Split train and test sets

In [24]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [26]:
y_train

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-tim',
  'O',
  'B-gpe',
  'B-per',
  'I-per',
  'I-per',
  'I-per',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'B-org',
  'I-org',
  'I-org',
  'O'],
 ['O',
  'O',
  'B-org',
  'I-org',
  'O',
  'B-per',
  'I-per',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-art',
  'I-art',
  'I-art',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'B-tim',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'B-per',
  'I-per',
  'O',
  'O',
  'O',
 

### Train a CRF model

In [27]:
X_train[0:5]

[[{'bias': 1.0,
   'word.lower()': 'the',
   'word[-3:]': 'The',
   'word[-2:]': 'he',
   'word.isupper()': False,
   'word.istitle()': True,
   'word.isdigit()': False,
   'postag': 'DT',
   'postag[:2]': 'DT',
   'BOS': True,
   '+1:word.lower()': 'spokesman',
   '+1:word.istitle()': False,
   '+1:word.isupper()': False,
   '+1:postag': 'NN',
   '+1:postag[:2]': 'NN'},
  {'bias': 1.0,
   'word.lower()': 'spokesman',
   'word[-3:]': 'man',
   'word[-2:]': 'an',
   'word.isupper()': False,
   'word.istitle()': False,
   'word.isdigit()': False,
   'postag': 'NN',
   'postag[:2]': 'NN',
   '-1:word.lower()': 'the',
   '-1:word.istitle()': True,
   '-1:word.isupper()': False,
   '-1:postag': 'DT',
   '-1:postag[:2]': 'DT',
   '+1:word.lower()': 'says',
   '+1:word.istitle()': False,
   '+1:word.isupper()': False,
   '+1:postag': 'VBZ',
   '+1:postag[:2]': 'VB'},
  {'bias': 1.0,
   'word.lower()': 'says',
   'word[-3:]': 'ays',
   'word[-2:]': 'ys',
   'word.isupper()': False,
   'word.is

In [28]:
# y_train[0:10]

In [29]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [30]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [31]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [32]:
trainer.train(path_export + '/ner_v1.crfsuite')

In [33]:
trainer.logparser.last_iteration

{'num': 50,
 'scores': {},
 'loss': 823.996243,
 'feature_norm': 29.018621,
 'error_norm': 19.348374,
 'active_features': 634,
 'linesearch_trials': 1,
 'linesearch_step': 1.0,
 'time': 0.009}

In [34]:
print(len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

50 {'num': 50, 'scores': {}, 'loss': 823.996243, 'feature_norm': 29.018621, 'error_norm': 19.348374, 'active_features': 634, 'linesearch_trials': 1, 'linesearch_step': 1.0, 'time': 0.009}


## Make predictions

To use the trained model, create pycrfsuite.Tagger, open the model and use "tag" method:

In [35]:
tagger = pycrfsuite.Tagger()
tagger.open(path_export + '/ner_v1.crfsuite')

Let's tag a sentence to see how it works:

In [36]:
# X = [sent2features(s) for s in sentences]
# y = [sent2labels(s) for s in sentences]

In [37]:
example_sent = X_test[0]
print("Predicted:", ' '.join(tagger.tag(example_sent)))

Predicted: O O O O O O O O O O O O O O O O O O O O


## Evaluate the model

In [38]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

Predict entity labels for all sentences in our testing set ('testb' Spanish data):

In [39]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

CPU times: user 29.2 ms, sys: 1.29 ms, total: 30.5 ms
Wall time: 29.5 ms


..and check the result. Note this report is not comparable to results in CONLL2002 papers because here we check per-token results (not per-entity). Per-entity numbers will be worse.  

In [41]:
# print(bio_classification_report(y_test, y_pred))

## Let's check what classifier learned

In [42]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
B-eve  -> I-eve   5.862141
B-org  -> I-org   5.394498
I-art  -> I-art   5.385130
B-art  -> I-art   5.310209
B-per  -> I-per   5.247143
I-org  -> I-org   4.573697
I-per  -> I-per   4.309017
I-eve  -> I-eve   4.290027
I-tim  -> I-tim   4.289169
B-gpe  -> I-gpe   3.966887
B-tim  -> I-tim   3.802855
B-geo  -> I-geo   3.249607
O      -> O       2.269199
I-geo  -> I-geo   2.240215
O      -> B-art   1.686230

Top unlikely transitions:
B-org  -> O       0.200442
B-geo  -> O       0.130459
B-art  -> O       0.059909
O      -> B-per   0.001346
B-gpe  -> B-gpe   -0.003576
B-tim  -> B-gpe   -0.007576
O      -> I-eve   -0.089257
I-per  -> O       -0.164576
O      -> I-gpe   -0.198555
B-geo  -> I-per   -0.287184
O      -> I-geo   -0.377052
O      -> I-per   -0.394023
O      -> I-art   -0.688711
O      -> I-tim   -1.537479
O      -> I-org   -2.151715


We can see that, for example, it is very likely that the beginning of an organization name (B-ORG) will be followed by a token inside organization name (I-ORG), but transitions to I-ORG from tokens with other labels are penalized. Also note I-PER -> B-LOC transition: a positive weight means that model thinks that a person name is often followed by a location.

Check the state features:

In [43]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
7.429995 B-tim  word[-2:]:0s
6.916157 O      bias
3.944121 B-tim  word[-3:]:day
3.647705 O      word[-2:]:ic
3.619117 B-tim  word[-2:]:ay
3.457040 O      BOS
3.329687 B-tim  word[-3:]:ber
2.914390 B-tim  -1:word.lower():in
2.911403 B-gpe  word.istitle()
2.762741 B-org  word.isupper()
2.753727 B-org  -1:word.lower():u.s.
2.676404 B-tim  word.isdigit()
2.622204 B-gpe  postag:JJ
2.542661 B-geo  -1:word.lower():in
2.531022 B-org  word[-3:]:ban
2.524125 I-gpe  word[-3:]:can
2.396443 B-gpe  word[-2:]:na
2.253424 B-per  word[-2:]:r.
2.176587 I-tim  word.isdigit()
2.118789 B-gpe  word.lower():north

Top negative:
-0.747706 O      word[-3:]:of
-0.780633 O      +1:word.lower():was
-0.781473 O      word[-3:]:ish
-0.813604 O      -1:word.lower():the
-0.837661 B-tim  +1:postag[:2]:NN
-0.864859 O      word[-2:]:sh
-0.892807 I-tim  word.istitle()
-0.898955 B-gpe  -1:postag[:2]:NN
-0.985283 B-gpe  word[-3:]:ean
-1.021609 B-gpe  +1:postag:NN
-1.188226 I-per  bias
-1.261623 B-gpe  word[-3:

In [ ]:
test = sentences[0]
test

In [ ]:
sent2features(test)